<a href="https://colab.research.google.com/github/Narusi/My-Projects/blob/master/ss%20dzivokli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [0]:
def FlatDetails(link):
    import requests
    from bs4 import BeautifulSoup

    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    date = ''    
    for x in soup.find_all('td',"msg_footer",):
        if x.text.find('Date') >= 0:
            date = (x.text[6:])    
    return date

def GetProperties(path, district='Unknown', pages=10, result='prices'):
    import csv
    
    #GET ALL PROPERTIES DISTRICT BY DISTRICT
    properties = []
    deals = ['hand_over/', 'sell/']
    postID = []
    links = {}
    
    invalid_items = []
    
    try:
        for dd in deals:
            if dd == 'sell/':
                tradeType = 'SELL'
            else:
                tradeType = 'RENT'

            for n in range(1, pages):
                
                if n == 1:
                    fullPath = path + dd
                else:
                    fullPath = path + dd + 'page' + str(n) + '.html'
                
                #print('Reading: ' + fullPath)
                r = requests.get(fullPath)
                soup = BeautifulSoup(r.text, 'html.parser')

                raw = soup.find_all('tr')
                #print('Found {} entries in this page'.format(len(raw)))
                for rr in raw:
                    temp = []

                    for ss in rr.find_all('td'):
                        temp.append(ss.text)
                    
                    #print(len(temp))
                    try:
                        if len(temp) == 10 and \
                        str(rr.get('id') + temp[3] + temp[4] + temp[5] + \
                            temp[6] + temp[7]) not in postID:
                            
                            if str(temp[8]) == '-':
                                sqmPrice = 0.00
                            else:
                                sqmPrice = float(str(temp[8]).replace(' €','').replace(',',''))
                            
                            if len(temp[9])<2 and temp[9]=='-':
                                price = 0.00
                            elif str(temp[9]).find('/')<0 and temp[9]!='buy ':
                                price = float(str(temp[9]).replace(' €','').replace(',','')\
                                            .encode("ascii", "ignore"))
                            else:
                                if str(temp[9]).find('/day')<0 and\
                                str(temp[9]).find('/week')<0\
                                and temp[9]!='buy ':
                                    price = float(str(temp[9]).replace('/mon.', '')\
                                                .replace(' €','').replace(',','')\
                                                .encode("ascii", "ignore"))
                                else:
                                    price = 0.00
                            
                            if tradeType == 'RENT':    
                                altPrice = float(price) * 120
                                altSQMprice = float(sqmPrice) * 120
                            else:
                                altPrice = price
                                altSQMprice = sqmPrice
                                                    
                            for x in rr.find_all('a'):
                                links[rr.get('id')] = 'https://www.ss.com/' +\
                                str(x.get('href'))
                            
                            #print(district, streetName, altPrice, altSQMprice)
                            if len(str(temp[4])) > 2 or len(str(temp[4])) == 0:
                                rooms = 0
                            elif len(str(temp[4])) > 0:
                                rooms = int(temp[4])
                            
                            size = temp[5]
                            floor = str(temp[6]).split('/')[0]
                            maxFloor = str(temp[6]).split('/')[-1]
                            project = str(temp[7])

                            if len(temp[3].split(' ')[0])>3:
                                streetName =  temp[3].split(' ')[0]
                                streetNo = temp[3].split(' ')[-1]
                            elif len(temp[3].split(' ')) > 2:
                                streetName =  str(temp[3].split(' ')[0]) + \
                                str(temp[3].split(' ')[1])
                                streetNo = temp[3].split(' ')[-1]
                            else:
                                streetName =  str(temp[3])
                                streetNo = ''                              
                            
                            postDate = FlatDetails(links[rr.get('id')])
                            comment = str(temp[2])
                            pID = str(str(rr.get('id'))[4:] + temp[3] + temp[4] + \
                                    temp[5] + temp[6] + temp[7])
                            
                            properties.append([pID, tradeType, comment, links[rr.get('id')], \
                                            district, streetName, streetNo, rooms, size, \
                                            floor, maxFloor, project, postDate, \
                                            sqmPrice, altSQMprice, price, altPrice])
                            
                            postID.append(str(rr.get('id') + temp[3] + temp[4] + \
                                            temp[5] + temp[6] + temp[7]))
                        else:
                            invalid_items.append(temp)
                    except:
                        invalid_items.append(temp)
    finally:
        with open('JUNK.csv', 'w', newline='', encoding='utf-16') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=';', quoting=csv.QUOTE_MINIMAL)
            for item in invalid_items:
                #Special characters would normaly crash 
                #any string manipulation so we need to encode them:
                csvwriter.writerow(str(item).encode("ascii", "ignore"))
        
        postID = []
        
    return properties

In [0]:
districts = {}

#GET ALL DISTRICTS AND THEIR LINKS
r = requests.get('https://www.ss.com/en/real-estate/flats/riga/')
soup = BeautifulSoup(r.text, 'html.parser')

raw = soup.find_all('a','a_category')
for r in raw:
    districts[r.text] = 'https://www.ss.com' + r.get('href')
    #print(r.text)

In [0]:
rajoni = ['Centre','Grizinkalns','Sarkandaugava','VEF','Teika']

cols = ['ID', 'Deal Type', 'Comment', 'Link', 'District', 'Street Name',
        'Street No.', 'Rooms', 'Size','Floor', 'Max. Floor','Project',
        'Post Date', 'Price of sqm', 'Alt. Price of sqm','Total Price', 'Alt. Price']

numCols = ['Rooms', 'Size','Floor', 'Max. Floor','Price of sqm',
           'Alt. Price of sqm','Total Price', 'Alt. Price']

In [41]:
temp = []

for r in rajoni:
    print('Processing ', r)
    tData = GetProperties(districts[r], district = r)
    for tt in tData:
        count = 0
        for t in tt:
            if t != '':
                count += 1
                
        if count == len(tt):
            temp.append(tt)

ipasumi = pd.DataFrame(temp, columns=cols).dropna()
ipasumi = ipasumi[ipasumi['Total Price'] > 0.0]

Processing  Centre
Processing  Grizinkalns
Processing  Sarkandaugava
Processing  VEF
Processing  Teika


In [42]:
for col in numCols:
    try:
        ipasumi[col] = pd.to_numeric(ipasumi[col])
    except:
        print(col)
        print(ipasumi[col].to_list())

ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])

selectedCols = ['Rooms', 'Size','Floor', 'Max. Floor','Total Price']

#AVG price per Rooms, Floors, Deal Type and Projects
avgRFDP = ipasumi.groupby(['Rooms','Floor','District','Project',
                           'Deal Type'])['Price of sqm'].median()
ipasumi['Discount'] = 0.0
for row in avgRFDP.index:
    for i in ipasumi.index:
        if ipasumi.loc[i, 'Rooms'] == list(row)[0] and \
        ipasumi.loc[i, 'Floor'] == list(row)[1] and \
        ipasumi.loc[i, 'District'] == list(row)[2] and \
        ipasumi.loc[i, 'Project'] == list(row)[3] and \
        ipasumi.loc[i, 'Deal Type'] == list(row)[4]:
            ipasumi.loc[i, 'Discount'] = 1 - ipasumi.loc[i, 'Price of sqm']/avgRFDP[row]

for raj in rajoni:
    print('\n',raj)
    print(ipasumi[np.logical_and(ipasumi['District'] == raj,
                                 ipasumi['Deal Type'] == 'RENT')][selectedCols].describe())   
    discounted = ipasumi[np.logical_and(ipasumi['Discount'] > 0.0, 
                                        ipasumi['District'] == raj)]
    discounted = discounted[discounted['Deal Type'] == 'RENT']

    if len(discounted.index) > 1:
        print('\nRelatively cheap trades:')
        print(discounted[['Link','Discount']].sort_values(by='Discount', 
                                                          ascending=False))

selectedCols = ['District','Street Name','Project','Post Date','Discount'] + selectedCols
renting = ipasumi[np.logical_and(ipasumi['Deal Type'] == 'RENT',
                                 ipasumi['Size'] >= 80)][selectedCols]
renting = renting[np.logical_and(renting['Rooms'] >= 3,
                                 renting['Floor'] != renting['Max. Floor'])]

renting[renting['Discount'] > 0.01].sort_values(by='Discount', ascending=False)


 Centre
            Rooms        Size       Floor  Max. Floor  Total Price
count  242.000000  242.000000  242.000000  242.000000   242.000000
mean     2.161157   62.070248    3.293388    5.599174   574.400826
std      1.040098   40.064979    1.655081    2.575112   418.649380
min      1.000000    5.000000    1.000000    1.000000   100.000000
25%      1.000000   32.250000    2.000000    5.000000   322.500000
50%      2.000000   55.000000    3.000000    5.000000   500.000000
75%      3.000000   80.000000    4.000000    6.000000   700.000000
max      6.000000  260.000000   14.000000   24.000000  2900.000000

Relatively cheap trades:
                                                  Link  Discount
72   https://www.ss.com//msg/en/real-estate/flats/r...  0.919800
194  https://www.ss.com//msg/en/real-estate/flats/r...  0.865261
124  https://www.ss.com//msg/en/real-estate/flats/r...  0.586000
43   https://www.ss.com//msg/en/real-estate/flats/r...  0.478750
243  https://www.ss.com//msg/en/real-

,District,Street Name,Project,Post Date,Discount,Rooms,Size,Floor,Max. Floor,Total Price
43,Centre,Chaka,Recon.,2020-08-01 11:49:00,0.478750,3,96,2.0,5,400.0
243,Centre,Stabu,Recon.,2020-02-01 21:29:00,0.464803,3,97,3.0,6,520.0
188,Centre,Mikela,Spec. pr.,2020-04-01 18:10:00,0.258441,3,96,3.0,5,580.0
149,Centre,Pulkv.,Recon.,2020-06-01 10:02:00,0.222500,3,95,2.0,5,591.0
263,Centre,Antonijas,Recon.,2020-02-01 10:30:00,0.176250,3,88,5.0,6,580.0
30,Centre,Merkela,Perewar,2020-08-01 14:11:00,0.137892,4,143,3.0,4,1100.0
165,Centre,Gertrudes,Perewar,2020-05-01 19:48:00,0.117446,3,84,2.0,6,650.0
2,Centre,Grostonas,New,2020-09-01 06:56:00,0.100934,3,89,6.0,8,900.0
87,Centre,Valdemara,Perewar,2020-07-01 14:13:00,0.094499,3,81,3.0,6,780.0
223,Centre,Vilandes,Recon.,2020-03-01 13:10:00,0.087264,4,155,2.0,6,1200.0


In [43]:
for key in districts:    
    if key != 'All announcements':
        tData = GetProperties(districts[key], district = key, pages=100)
        for ii in tData:
            temp.append(ii)
        print(key)

ipasumi = pd.DataFrame(temp, columns=cols).dropna()

Centre
Agenskalns
Aplokciems
Beberbeki
Bergi
Bierini
Bolderaya
Breksi
Bukulti
Bulli
Chiekurkalns
Darzciems
Daugavgriva
Dreilini
Dzeguzhkalns
Grizinkalns
Ilguciems
Imanta
Janjavarti
Jaunciems
Jaunmilgravis
Katlakalns
Kengarags
Kipsala
Kleisti
Kliversala
Krasta st. area
Kremeri
Kundzinsala
Mangali
Mangalsala
Maskavas priekshpilseta
Mezhapark
Mezhciems
Plyavnieki
Purvciems
Sarkandaugava
Shampeteris-Pleskodale
Shkirotava
Stacija-Tirgus
Teika
Tornjakalns
Vecaki
Vecdaugava
Vecmilgravis
Vecriga
Voleri
Yugla
Zakusala
Zasulauks
Ziepniekkalns
Zolitude
VEF
Other


In [44]:
ipasumi = ipasumi[np.logical_and(ipasumi['Total Price'] > 0,
                                 ipasumi['Rooms'] != 'Other')]
for col in numCols:
    ipasumi[col] = pd.to_numeric(ipasumi[col])

ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])
ipasumi = ipasumi.dropna()
ipasumi.info()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4103 entries, 0 to 4283
Data columns (total 17 columns):
ID                   4103 non-null object
Deal Type            4103 non-null object
Comment              4103 non-null object
Link                 4103 non-null object
District             4103 non-null object
Street Name          4103 non-null object
Street No.           4103 non-null object
Rooms                4103 non-null int64
Size                 4103 non-null int64
Floor                4103 non-null float64
Max. Floor           4103 non-null int64
Project              4103 non-null object
Post Date            4103 non-null datetime64[ns]
Price of sqm         4103 non-null float64
Alt. Price of sqm    4103 non-null float64
Total Price          4103 non-null float64
Alt. Price           4103 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(3), object(8)
memory usage: 577.0+ KB


In [45]:
def getDiscounts(ipasumi):
    #Median prices to estimate fair prices
    filterOne = ['Rooms','Floor','Max. Floor','District','Project','Deal Type','Street Name']
    filterTwo = ['Rooms','Floor','Max. Floor','District','Project','Deal Type']
    filterThree = ['Floor','District','Project','Deal Type']

    fltOne = ipasumi.groupby(filterOne)['Price of sqm'].median()
    fltTwo = ipasumi.groupby(filterTwo)['Price of sqm'].median()
    fltThree = ipasumi.groupby(filterThree)['Price of sqm'].median()

    fltOneC = ipasumi.groupby(filterOne)['Price of sqm'].count()
    fltTwoC = ipasumi.groupby(filterTwo)['Price of sqm'].count()
    fltThreeC = ipasumi.groupby(filterThree)['Price of sqm'].count()

    ipasumi['Discount'] = 0.0
    for row in fltOne.index:
        if fltOneC[row] > 5:
            for i in ipasumi.index:
                if ipasumi.loc[i, filterOne[0]] == list(row)[0] and \
                ipasumi.loc[i, filterOne[1]] == list(row)[1] and \
                ipasumi.loc[i, filterOne[2]] == list(row)[2] and \
                ipasumi.loc[i, filterOne[3]] == list(row)[3] and \
                ipasumi.loc[i, filterOne[4]] == list(row)[4] and \
                ipasumi.loc[i, filterOne[5]] == list(row)[5] and \
                ipasumi.loc[i, filterOne[6]] == list(row)[6]:
                    ipasumi.loc[i, 'Discount'] = 1 - ipasumi.loc[i, 'Price of sqm']/fltOne[row]

    for row in fltTwo.index:
        if fltTwoC[row] > 5:
            for i in ipasumi.index:
                if ipasumi.loc[i, filterTwo[0]] == list(row)[0] and \
                ipasumi.loc[i, filterTwo[1]] == list(row)[1] and \
                ipasumi.loc[i, filterTwo[2]] == list(row)[2] and \
                ipasumi.loc[i, filterTwo[3]] == list(row)[3] and \
                ipasumi.loc[i, filterTwo[4]] == list(row)[4] and \
                ipasumi.loc[i, filterTwo[5]] == list(row)[5]:
                    ipasumi.loc[i, 'Discount'] = 1 - ipasumi.loc[i, 'Price of sqm']/fltTwo[row]

    for row in fltThree.index:
        if fltThreeC[row] > 5:
            for i in ipasumi.index:
                if ipasumi.loc[i, filterThree[0]] == list(row)[0] and \
                ipasumi.loc[i, filterThree[1]] == list(row)[1] and \
                ipasumi.loc[i, filterThree[2]] == list(row)[2] and \
                ipasumi.loc[i, filterThree[3]] == list(row)[3]:
                    ipasumi.loc[i, 'Discount'] = 1 - ipasumi.loc[i, 'Price of sqm']/fltThree[row]
    return ipasumi

ipasumi = getDiscounts(ipasumi)
ipasumi[ipasumi['Discount'] > 0.01]

,ID,Deal Type,Comment,Link,District,Street Name,Street No.,Rooms,Size,Floor,Max. Floor,Project,Post Date,Price of sqm,Alt. Price of sqm,Total Price,Alt. Price,Discount
1,6795690Turgeneva 21a2552/6Perewar,RENT,Izīrē par 350 Eur+Komunālie maksājumi (nesmēķē...,https://www.ss.com//msg/en/real-estate/flats/r...,Centre,Turgeneva,21a,2,55,2.0,6,Perewar,2020-09-01 09:33:00,6.36,763.2,350.0,42000.0,0.316497
5,7099979Lachplesha 1272953/3Recon.,RENT,"Īpašnieks izīrē modernu, gaišu un ļoti mājīgu ...",https://www.ss.com//msg/en/real-estate/flats/r...,Centre,Lachplesha,127,2,95,3.0,3,Recon.,2020-08-01 22:18:00,8.95,1074.0,850.0,102000.0,0.131911
6,7322911Maskavas 761303/4Perewar,RENT,Īpašnieks izīrē saulainu un siltu dzīvoklīti K...,https://www.ss.com//msg/en/real-estate/flats/r...,Centre,Maskavas,76,1,30,3.0,4,Perewar,2020-08-01 20:04:00,7.00,840.0,210.0,25200.0,0.239130
8,7322720Chaka 593855/5Perewar,RENT,Īpašnieks izīrē plašu 3 -istabu dzīvokli 85 kv...,https://www.ss.com//msg/en/real-estate/flats/r...,Centre,Chaka,59,3,85,5.0,5,Perewar,2020-08-01 19:21:00,5.88,705.6,500.0,60000.0,0.193416
15,7321914Dzirnavu 662806/7Perewar,RENT,"Dzirnavu iela, 450 € + komunālie maksājumi. \r...",https://www.ss.com//msg/en/real-estate/flats/r...,Centre,Dzirnavu,66,2,80,6.0,7,Perewar,2020-08-01 16:36:00,5.63,675.6,450.0,54000.0,0.163447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4180,7190150Kaplavas 52852/6New,SELL,Pieejams plašs dzīvoklis ar pilnu iekšējo apda...,https://www.ss.com//msg/en/real-estate/flats/r...,Ziepniekkalns,Kaplavas,5,2,85,2.0,6,New,2019-12-23 14:29:00,1071.00,1071.0,91000.0,91000.0,0.259848
4203,6742546Kaplavas 53994/6New,SELL,Tiek piedāvāts plašs un stilīgs dzīvoklis jaun...,https://www.ss.com//msg/en/real-estate/flats/r...,Ziepniekkalns,Kaplavas,5,3,99,4.0,6,New,2019-12-12 23:44:00,1313.00,1313.0,130000.0,130000.0,0.074048
4234,6956031Apuzes 51a2761/3New,SELL,Дом А сдан в эксплуатацию в продаже остались п...,https://www.ss.com//msg/en/real-estate/flats/r...,Zolitude,Apuzes,51a,2,76,1.0,3,New,2020-08-01 15:45:00,1285.00,1285.0,97680.0,97680.0,0.056535
4240,7140327Priedaines 132721/4New,SELL,Лучшее ценовое предложение в секторе новых про...,https://www.ss.com//msg/en/real-estate/flats/r...,Zolitude,Priedaines,13,2,72,1.0,4,New,2020-07-01 15:26:00,1181.00,1181.0,85000.0,85000.0,0.132893


In [51]:
#Dropping comment as csv cannot properly store long comment fields:
ipasumi.drop('Comment', axis=1).to_csv('ssDzivokli.csv', index=False, sep = ';', 
                                       decimal = ',', encoding='utf=16')

altPrice = ipasumi['Alt. Price of sqm'].where(ipasumi['Project']=='New').mean()
print('Mean price for square meter in New project flat: {} eiro'.format(round(altPrice,2)))

Mean price for square meter in New project flat: 2216.88 eiro


In [0]:
try:
    conn = sqlite3.connect('miniSS.db')

    try:
        data = pd.read_sql('SELECT * FROM PropertiesRAW', \
                           conn, index_col='ID').drop('index', axis = 1)

        ipasumi = ipasumi.append(data).sort_values(by=['Post Date'], ascending=False)
        ipasumi = ipasumi.drop_duplicates(['District', 'Street Name', \
                                           'Street No.', 'Rooms', 'Size',\
                                           'Floor', 'Max. Floor','Project'])
        ipasumi = getDiscounts(ipasumi)
        conn.cursor().execute('DROP TABLE PropertiesRAW')
        conn.commit()
        ipasumi.to_sql('PropertiesRAW', conn)
    except:
        ipasumi = getDiscounts(ipasumi)
        ipasumi.to_sql('PropertiesRAW', conn)

    conn.close
    
    ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])
    ipasumi.to_csv('data.csv', index=False, sep = ';', decimal = ',',
                   encoding='utf-16')
except:
    ipasumi['Post Date'] = pd.to_datetime(ipasumi['Post Date'])
    ipasumi = getDiscounts(ipasumi)
    ipasumi.to_csv('data.csv', index=False, sep = ';', decimal = ',', 
                   encoding='utf-16')

In [0]:
#pd.DataFrame(GetProperties(districts[rajoni], district = rajoni, pages=100),
#             columns=cols)